# Project Report (Water Potability Classification)
## Training a classification model to determine water potability based on measurable predictors



### Introduction

Potable water (water that is safe to drink) is a basic requirement for human life. Lack of access to potable water is an issue that continues to affect many regions globally. The United Nations Sustainable Development Goal 6 is accordingly: "Ensure availability and sustainable management of water and sanitation for all”. According to the most recent 2020 report from the World Health Organization (WHO), an estimated 26% of the global population (1.97 billion individuals) did not have access to safely managed drinking water services (1). Researching sufficiently fast, reliable, and cost-effective ways of evaluating the safety of drinking water is crucial to addressing this issue. In 2016, UNICEF called for the production of rapid-test kits for detecting the bacterium *Escherichia coli* (usually present in fecally contaminated water) (https://www.unicef.org/innovation/rapid-water-quality-testing). Apart from microbes, various other factors affect water potability. For example, trihalomethanes are a type of disinfection by-products produced during some water treatment processes, and have been demonstrated to be carcinogenic to humans (2).

In this project, we will utilize a synthetically generated dataset (https://www.kaggle.com/datasets/adityakadiwal/water-potability) containing over 3000 observations, each representing a sample of water. Each observation contains nine variables that affect water potability (pH, hardness, total dissolved solids, chloramines, sulfate, conductivity, organic carbon, trihalomethanes, and turbidity) as well as a score indicating whether or not the water is potable. Using this dataset, we aim to demonstrate how a classification model can be trained to accurately evaluate whether or not an unknown sample of water is potable based on measurable variables.

### Methods & Results

All statistical methods are referenced from Data Science: A First Introduction by Tiffany Timbers, Trevor Campbell, and Melissa Lee (3).

#### Exploratory Data Analysis

In [ ]:
library(tidyverse)
library(repr)
library(tidymodels)
library(GGally)
library(ISLR)

ERROR: ignored

In [ ]:
set.seed(1999)
options(repr.plot.width = 8, repr.plot.height = 7)
#1. READ DATA
url <- "https://raw.githubusercontent.com/Swapnil-Dubey/Dsci100_Group93_/main/data/water_potability.csv"
water_potability_raw <- read_csv(url) #unedited "raw" dataset

print(paste("Number of rows in the table: ",nrow(water_potability_raw))) #show number of observations in the raw dataset.
print(paste("Number of columns in the table: ",ncol(water_potability_raw))) #show number of variables in the raw dataset.

head(water_potability_raw)

Rows: 3276 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (10): ph, Hardness, Solids, Chloramines, Sulfate, Conductivity, Organic_...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "Number of rows in the table:  3276"
[1] "Number of columns in the table:  10"


ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NA,204.8905,20791.32,7.300212,368.5164,564.3087,10.379783,86.99097,2.963135,0
3.716080,129.4229,18630.06,6.635246,NA,592.8854,15.180013,56.32908,4.500656,0
8.099124,224.2363,19909.54,9.275884,NA,418.6062,16.868637,66.42009,3.055934,0
8.316766,214.3734,22018.42,8.059332,356.8861,363.2665,18.436524,100.34167,4.628771,0
9.092223,181.1015,17978.99,6.546600,310.1357,398.4108,11.558279,31.99799,4.075075,0
5.584087,188.3133,28748.69,7.544869,326.6784,280.4679,8.399735,54.91786,2.559708,0


<center> (Figure 1: Original water potability dataset) </center>

<hr noshade>

Before we move on to the classification component, it is always required that we take a look at our data and understand the problem and the structure of the data and clean the data if necessary. 
    <br> </br>
    
Here you can notice that the predictive variable i.e <i>Potability</i> is of numeric type, however it only takes up the value 0 or 1 and hence we use `as.factor()` to convert it into a factor. <br>
Furthermore, we don't want our data to contain any `NA` values, and hence we will ignore those values using `na.omit()`

In [ ]:
#2. CLEAN DATA AND REMOVE NA ROWS
water_potability_clean <- water_potability_raw |>
    rename(hardness = Hardness, solids = Solids, chloramines = Chloramines, sulfate = Sulfate, conductivity = Conductivity, organic_carbon = Organic_carbon, trihalomethanes = Trihalomethanes, turbidity = Turbidity, potability = Potability) |>
    mutate(potability = as_factor(potability)) |>
    na.omit()

print(paste("Number of rows in the table: ",nrow(water_potability_clean))) #number of observations in the cleaned dataset (NA removed)
print(paste("Number of columns in the table: ",ncol(water_potability_clean))) #number of variables in the cleaned dataset (NA removed)

head(water_potability_clean)

<center> (Figure 2: Clean water potability dataset) </center>

<hr noshade>

The next step is to divide the data into training and testing datasets after selecting a predictive question to answer and wrangling the data. We split it in the ratio 3:1 (training: 75%, testing: 25%) 
    
The `initial_split` function from `tidymodels` library splits the data for us.
    

In [ ]:
set.seed(1999)
#3. CREATE TRAINING DATA and TESTING DATA
water_split <- water_potability_clean |> #split the data 0.75/0.25
    initial_split(prop = 0.75, strata = potability) 


water_train <- training(water_split)  #set the 0.75 split as the training data
water_test <- testing(water_split) #set the 0.25 split as the testing data

head(water_train)

<center> (Figure 3: Training water potability dataset) </center>

<hr noshade> 

Here we are trying to analyze the data i.e finding the mean of all the predictor variables for each group (potable and non-potable) <br>

We use `summarize()` and `group_by` to find the means of all the predictor variables and count the number of observations for each group. Note that 59.7% of the observations in the training dataset are classified as not potable (0) and the remaining 40.3% are classified as potable (1).
   

In [ ]:
#4. SUMMARIZE DATA IN A TABLE
water_train_means <- water_train |> #find means for all predictors grouped by potability
    group_by(potability) |>
    summarize(across(ph:turbidity, mean, na.rm=TRUE))

water_train_nrows <- water_train |> #count number of observations in the two groups
    group_by(potability) |>
    summarize(n = n())

water_train_summary <- water_train_means |> #merge grouped n dataframe with grouped means dataframe
    merge(water_train_nrows)

water_train_summary

water_train %>%
    group_by(potability) %>%
    summarize(percentage = n()/nrow(water_train))

<center> (Figure 4: Summarizing our water potability dataset to <br> find the mean of all the variables for each potability value) </center>

<hr noshade>

Let us get a summary of our data using the `summary()` function. This gives us the minimum, maximum, mean, median and, quantile values. <br>
Quantile value determines what a fraction of the data looks like. For example, the 1st quantile value for ph tells us that 25% of the data lies below 6.0767. <br>
<i>Note:</i> We do not remove the outliers from our data because they hold meaning. For example, a value as low as 320.9 in solids isn't considered an outlier because it corresponds to less number of minerals present in that particular sample of water.

In [ ]:
summary_water_potability <- summary(water_train)
summary_water_potability

<center> (Figure 5: Summary of our water potability dataset) </center>

<hr noshade>

We now want to check for the number of predictor variables necessary for our model. For that purpose, we find the correlation between each predictor variable. <br>
The `ggpairs()` function from `GGally` library helps us do so by visualizing the correlation between them. <br>
Here we find that none of the predictor variables are strongly related to each other and hence we decide to consider all the variables when building our model. The observations are also color-coded according to class. We can observe here that the potable and non-potable data points are virtually indistinguishable and share highly similar box plot distributions.<br>
<i>Note:</i> Weak correlation corresponds to a value closer to 0 while a strong correlation has a correlation value closer to 1 or -1.

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 20)

water_eda <- water_train |>
    ggpairs(aes(color = potability), 
            title = "Correlation of the variables in water potability \n",
            upper = list(continuous = wrap("cor", size = 5))) +  
    theme(text = element_text(size = 17))

water_eda

<center> (Figure 6: Correlation of the variables in water potability) </center>

<hr noshade>

Whenever using K-nearest neighbor classification scaling matters (size and range of each variable). This is because any variables with a large scale will have a much larger effect than variables with a small scale. After all, the classifier predicts classes by identifying observations closest to them. <br>
We normally scale the data when creating the knn-model recipe using `step_scale()` and `step_center()`.

Here, we scale all of the predictors as a percentage of the range of each predictor. This is done according to (Xi - Xmin)/(Xmax - Xmin). This is done for clear visualization of the means of each predictor according to class and to enable visual comparison across predictors.

In [ ]:
#5. SCALE TRAINING DATA (0 to 1)
min_table <- water_train |> #min value of all predictors
    select(-potability) |>
    map_df(min)

max_table <- water_train |> #max value of all predictors
    select(-potability) |>
    map_df(max)

diff_table <- max_table - min_table #max - min for all predictors

water_train_scaled <- water_train |> #scale all predictors X (0 to 1) according to (Xi - Xmin)/(Xmax - Xmin)
    select(-potability) |>
    mutate(ph = (ph - min_table$ph) / diff_table$ph, 
           hardness = (hardness - min_table$hardness) / diff_table$hardness,
           solids = (solids - min_table$solids) / diff_table$solids, 
           chloramines = (chloramines - min_table$chloramines) / diff_table$chloramines,
           sulfate = (sulfate - min_table$sulfate) / diff_table$sulfate, 
           conductivity = (conductivity - min_table$conductivity) / diff_table$conductivity,
           organic_carbon = (organic_carbon - min_table$organic_carbon) / diff_table$organic_carbon, 
           trihalomethanes = (trihalomethanes - min_table$trihalomethanes) / diff_table$trihalomethanes,
           turbidity = (turbidity - min_table$turbidity) / diff_table$turbidity) |>
    bind_cols(potability = water_train$potability)

Next, we will calculate the means for all predictors in both groups (potable and not potable).

In [ ]:
water_train_scaled_means_potable <- water_train_scaled |>
    filter(potability == "1") |>
    select(-potability) |>
    map_df(mean, na.rm = TRUE)
    

water_train_scaled_means_not_potable <- water_train_scaled |>
    filter(potability == "0") |>
    select(-potability) |>
    map_df(mean, na.rm = TRUE)
    

potability_tibble <- tibble(potability = 0:1) |>
    mutate(potability = as_factor(potability))

water_train_scaled_means <- bind_rows(water_train_scaled_means_not_potable, water_train_scaled_means_potable) |>
    bind_cols(potability_tibble)

water_train_scaled_means

<center> (Figure 7: Scaled means of all predictors according to potability) </center>

<hr noshade>

We visualize the means of each predictor variable grouped by each of the potability types to compare and analyze the trend <br>
Here we make use of the `ggplot()` function from the `ggplot2` package which has already been loaded by the `tidyverse` package. 

The percentage-scaled means of each predictor are plotted below in a bar graph, separated by potability. Observe that there is virtually no difference in the means of each predictor between the potable and non-potable groups. This indicates that the data set was generated randomly which will make generating an accurate classification model quite difficult.

In [ ]:
#6. PLOT SCALED TRAINING DATA (0-1) IN A BARPLOT SHOWING MEANS OF EACH PREDICTOR
options(repr.plot.width = 8, repr.plot.height = 8)

water_train_scaled_longer <- water_train_scaled_means |> #pivot longer, generating columns "predictor" and "value"
    pivot_longer(c(ph, hardness, solids, chloramines, sulfate, conductivity, organic_carbon, trihalomethanes, turbidity), names_to = "predictor")

water_train_barplot <- water_train_scaled_longer |> #barplot showing the scaled (0 to 1) means of each predictor grouped by potability
    mutate(potability = fct_recode(potability, "no" = "0", "yes" = "1")) |>
    ggplot(aes(y = reorder(predictor, value), x = value, fill = potability)) + 
    geom_col(position = 'dodge') + 
    xlab("Standardized value") +
    ylab("Predictor") +
    labs(fill = "Potable?") +
    ggtitle("Comparison of mean values of water potability predictors") +
    theme(text = element_text(size = 12))

water_train_barplot

<center> (Figure 8: Comparison of mean values of water potability predictors) </center>

<hr noshade>

To demonstrate this, below is a scatterplot comparing the distributions of two predictors (in this case, pH and hardness). These predictors were not selected for any particular reason other than visualization. We can observe that the distribution is quite random and there is a significant overlap between the potable and non-potable data points. While it is not shown here, the same is true using any combination of predictors.

In [ ]:
#7. SCALE TRAINING DATA USING SCALE()
 water_train_scaled_2 <- water_train |> #scale all predictors using the scale() function
     select(-potability) |>
     scale(center = TRUE, scale = TRUE) |>
     bind_cols(potability = water_train$potability)

#8. PLOT TRAINING DATA IN A SCATTERPLOT COMPARING THE DISTRIBUTION ACCORDING TO TWO SELECTED PREDICTORS
water_train_scatterplot <- water_train |> #scatterplot comparing two predictors--in this case, pH and hardness, standardized
    mutate(potability = fct_recode(potability, "no" = "0", "yes" = "1")) |>
    ggplot(aes(x = ph, y = hardness, color = potability)) +
    geom_point() +
    labs(x = "pH", y = "Hardness (mg/L)", color = "Potable?") +
    theme(text = element_text(size = 20))

water_train_scatterplot

<center> (Figure 9: Distribution of mean values of water potability predictors) </center>

#### Generating, training, and testing the classification model

From the exploratory data analysis, we made the following observations:
- This synthetically generated data set was randomly generated without adherence to bona fide thresholds that determine water potability (even by the description provided by the author of the data set). A demonstrable example is shown immediately above in Figure 9 above. Note that observations representing potable water (in blue) are present all across the pH spectrum, even at highly acidic (pH < 5) or highly basic (pH > 10) conditions. The same pattern can be observed by looking at the other predictors (not shown).
- One consequence of randomly generated data in this manner is that the predictors have a low correlation with one another. We will therefore proceed with the classification using all predictors. It is recommended that when using real data (and not synthetically generated data) to check for correlation between the predictors (multicollinearity).
- Another consequence is that the model generated in the following data analysis will have low accuracy and should not be used for water potability classification in the real world--only as a demonstration.

Next, we will proceed to generate, train, and test the classification model.  

First, we will generate the model for v-fold cross-validation and the recipe to classify potability based on all predictors. This is done using the `vfold_cv()` function from the `tidymodels` library. This process, known as cross-validation, divides the data set into ten folds and validates each of the folds against the others in sequence. This allows us to determine the best k-value by evaluating the accuracy of the classifier multiple times.

In [ ]:
set.seed(1999)

water_vfold <- vfold_cv(water_train, v = 10, strata = potability)

water_recipe <- recipe(potability ~ ph+hardness+sulfate, data = water_train) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors())

<hr noshade>

Next, we will tune our K-nearest neighbors specification to determine the K-value which will give us the highest accuracy. Here we generate a K-nearest neighbors model using the `nearest_neighbor()` function, set the engine to `kknn` (K-nearest neighbors), and the mode to `classification`. Then, using the `workflow()` function, we combine the recipe and model to tune our classifier.

In [ ]:
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
    set_engine("kknn") |>
    set_mode("classification")
 
knn_results <- workflow() |>
    add_recipe(water_recipe) |>
    add_model(knn_spec) |>
    tune_grid(resamples = water_vfold, grid = 10) |>
    collect_metrics()

<hr noshade>

By filtering the results of our workflow by the "accuracy" metric and ordering it by descending accuracy, we are presented with the K-value with the highest accuracy at the top of the table.

In [ ]:
accuracies <- knn_results |>
    filter(.metric == "accuracy") |>
    arrange(desc(mean))

accuracies

<center>(Figure 10: Table of accuracies at each K-value)</center>

<hr noshade>

We can also visualize the K-value selection process in the form of a line graph with accuracy on the y-axis vs. the k-value on the x-axis. Note that accuracy is at its maximum when k = 11 (given the high randomness of this data set, this value will vary greatly if the random seed is changed compared to a real-world data set).

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)

accuracy_vs_k <- ggplot(accuracies, aes(x = neighbors, y = mean)) +
    geom_point() +
    geom_line() +
    labs(x = "Neighbors", y = "Accuracy Estimate") + 
    ggtitle("Accuracy versus K-value") +
    theme(text = element_text(size = 12))

accuracy_vs_k

<center>(Figure 11: Accuracy versus K-value)</center>

<hr noshade>

We will then generate a new K-nearest neighbors model using K = 12.

In [ ]:
knn_best_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 12) |>
    set_engine("kknn") |>
    set_mode("classification")

<hr noshade>

Then, we can put the recipe generated earlier together with our new model in the `workflow()` function and fit it to our training data set. Our classification model is now trained and complete!

In [ ]:
water_fit <- workflow() |>
    add_recipe(water_recipe) |>
    add_model(knn_best_spec) |>
    fit(data = water_train)

<hr noshade>

Now, we can use test the model on the testing data set to determine how accurate it is:

In [ ]:
water_predictions <- predict(water_fit, water_test) |>
    bind_cols(water_test)

head(water_predictions)

<center>(Figure 12: Table of predictions of the testing data set.)</center>

<hr noshade>

Next, we can retrieve the accuracy of our predictions.

In [ ]:
water_acc <- water_predictions |>
    metrics(truth = potability, estimate = .pred_class) |>
    select(.metric, .estimate) |>
    head(1) |>
    pull()

water_acc

<hr noshade>

~ 68.19% of the time, the classification model correctly predicted whether the water was potable or not. This is quite a low value but to be expected given the randomness of this data. Most of the time, the values of the predictors beared no meaning on whether or not the water was potable. However, if the data set had been better generated, then the accuracy would be much higher.

Next, we can generate a confusion matrix to show how many false positives (incorrectly guessed that the water was potable when it was not) and false negatives (incorrectly guessed that the water was not potable when it was) there were.

In [ ]:
water_cm <- water_predictions |>
    conf_mat(truth = potability, estimate = .pred_class)

water_cm

<center>(Figure 13: Confusion matrix for water potability predictions)</center>

<hr noshade>

There were 242 true negatives, 114 false negatives, 58 true positives, and 89 false positives. Overall, these do not represent great numbers, and a classification model returning such values would not suffice for evaluating water potability. This can be attributed to either a poor classification model, or (more likely in this case) poor data quality.

### Discussion

As described previously, during our preliminary data analysis the dataset used was found to be highly random with a low correlation between the predictors. Additionally, when grouped by class (potable vs. non-potable), the means of all predictors were highly similar. We observe that approximately 60% of the observations were classified as potable. Thus, a 68.1% accuracy is optimal with the current database. We expect to see a much higher accuracy when a classification model is generated using our steps with a real-world dataset. 

Water is of major importance to all living organisms, especially humans, whose bodies are made up of 60% water. Lack of access to potable water is an ever-growing issue that continues to affect many regions globally. Using a classification model generated similarly to what we have done, we can recognize which water bodies are safe to drink based on their chemical and physical properties such as pH, conductivity, sulfate content, and what causes the variability in the potability. While our analysis was restricted to nine predictors (pH, hardness, total dissolved solids, chloramines, sulfate, conductivity, organic carbon, trihalomethanes, and turbidity), there are many other predictors which can and should be utilized for better classification. Future research could be done to determine which predictors are most analytically valuable and economically feasible. The impact of certain predictors will likely vary between the geographical region and should be well-researched before implementing any model in real-world use.

### References

(1) World Health Organization; United Nations Children’s Fund (UNICEF); World Bank. State of the World’s Drinking Water: An Urgent Call to Action to Accelerate Progress on Ensuring Safe Drinking Water for All; World Health Organization: Geneva, 2022.

(2) Srivastav, A. L.; Patel, N.; Chaudhary, V. K. Disinfection By-Products in Drinking Water: Occurrence, Toxicity and Abatement. Environmental Pollution 2020, 267, 115474. https://doi.org/10.1016/j.envpol.2020.115474.

(3) Timbers, T.-A.; Campbell, T.; Lee, M. Data Science: An Introduction, First edition.; Statistics; CRC Press: Boca Raton, 2022.